In [16]:
from pytube import YouTube
from typing import Tuple
from pytube.query import StreamQuery
import re

def is_valid_youtube_url(url: str) -> bool:
    if not isinstance(url, str):
        return False 
    pattern = r'^https://www\.youtube\.com/watch?v=[A-Za-z0-9_-]{11}$'  # youtube vido ids are always 11 chars long
    if "shorts" in url:
        pattern = r'^https://www\.youtube\.com/shorts/[A-Za-z0-9_-]{11}$'  # youtube vido ids are always 11 chars long
    return re.match(pattern, url) is not None


def get_yt_streams(url: str,
                   my_proxies: dict = {}) -> Tuple[YouTube, str, str, StreamQuery, StreamQuery, StreamQuery]:
    try:
        # validate url
        if is_valid_youtube_url(url):
            # load in video
            yt = YouTube(url,
                        proxies=my_proxies)
            
            # get title and thumbnail
            yt_title = yt.title
            yt_thumbnail_url = yt.thumbnail_url
            
            # audio only streams
            audio_only_streams = yt.streams.filter(file_extension="mp4",
                                                only_audio=True, 
                                                type="audio").order_by("abr").asc()

            # video only streams
            video_only_streams = yt.streams.filter(file_extension="mp4",
                                                only_video=True, 
                                                type="video").order_by("resolution").asc()
            
            # audio and video joint streams
            audio_video_streams = yt.streams.filter(file_extension="mp4", 
                                                    only_audio=False, 
                                                    only_video=False, 
                                                    progressive=True,
                                                    type="video").order_by("resolution").asc()
            
            return yt, yt_title, yt_thumbnail_url, audio_only_streams, video_only_streams, audio_video_streams
        else:
            raise ValueError(f"invalid input url: {url}")
    except Exception as e:
        raise ValueError(f"get_yt_streams failed with exception {e}")

In [61]:
import tempfile
from pytube import YouTube
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

def download_joint_stream(yt: YouTube,
                          itag: int,
                          save_dir: str,
                          yt_title: str) -> None:
    try:
        yt.streams.get_by_itag(itag).download(filename=save_dir + yt_title + ".mp4")
    except Exception as e:
        raise ValueError(f"download_joint_stream failed with exception {e}")
    
    
def download_separate_streams_and_join(yt: YouTube,
                                      audio_itag: int,
                                      video_itag: int,
                                      save_dir: str,
                                      yt_title: str):
    with tempfile.TemporaryDirectory() as tmpdirname:
        tmpaudiopath = tmpdirname + "/" + yt_title + "_audio.mp4"
        tmpvideopath = tmpdirname + "/" + yt_title + "_video.mp4"
        yt.streams.get_by_itag(audio_itag).download(filename=tmpaudiopath)
        yt.streams.get_by_itag(video_itag).download(filename=tmpvideopath)
        
        # combine the video clip with the audio clip
        video_clip = VideoFileClip(tmpvideopath)
        audio_clip = AudioFileClip(tmpaudiopath)
        video_clip.audio = audio_clip
        video_clip.write_videofile(
            save_dir + "/" +  yt_title + ".mp4",
            codec="libx264",
            audio_codec="aac",
            temp_audiofile="temp-audio.m4a",
            remove_temp=True,
        )

In [62]:
download_separate_streams_and_join(yt, 
                                    139,
                                    160, 
                                    main_dir,
                                    yt_title)

Moviepy - Building video /Users/jeremywatt/Desktop/content_creation/youtube_downloader/Can Costco Samples Make You Full?.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video /Users/jeremywatt/Desktop/content_creation/youtube_downloader/Can Costco Samples Make You Full?.mp4



Moviepy - Done !
Moviepy - video ready /Users/jeremywatt/Desktop/content_creation/youtube_downloader/Can Costco Samples Make You Full?.mp4


In [55]:
for f in video_only_streams:
    print(f)

<Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">
<Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">
<Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">
<Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">


In [49]:
video_only_choices

[('144p', 160),
 ('240p', 133),
 ('360p', 134),
 ('480p', 135),
 ('720p', 136),
 ('1080p', 137)]

In [50]:
audio_only_choices

[('48kbps', 139), ('128kbps', 140)]

In [ ]:
download_joint_stream(yt, 
                      18, 
                      main_dir,
                      yt_title)

In [47]:
url = "https://www.youtube.com/shorts/7OkZ0lQ-XCQ"

# get title, thumbnail_url, audio only streams, video only streams, and joint streams
yt, yt_title, yt_thumbnail_url, audio_only_streams, video_only_streams, audio_video_streams = get_yt_streams(url)

In [48]:
# grab all choices from different stream types
audio_only_choices = [(v.abr, v.itag) for v in audio_only_streams]
video_only_choices = [(v.resolution, v.itag) for v in video_only_streams]
audio_video_joint_choices = [(v.resolution, v.itag) for v in audio_video_streams]

In [37]:
audio_video_joint_choices

[('360p', 18), ('720p', 22)]

In [40]:
from youtube_downloader import main_dir

In [42]:
download_joint_stream(yt, 
                      18, 
                      main_dir,
                      yt_title)

# merge an audio and video file

In [ ]:
from pytube import YouTube

yt = YouTube('https://www.youtube.com/watch?v=DongWSk9rEs')
yt.streams.get_by_itag(139).download(filename="audio.mp4")
yt.streams.get_by_itag(137).download(filename="video.mp4")

Moviepy - Building video test.mp4.
MoviePy - Writing audio in temp-audio.m4a


MoviePy - Done.
Moviepy - Writing video test.mp4



t:  23%|██▎       | 22965/100240 [03:37<10:37, 121.23it/s, now=None]

KeyboardInterrupt: 